# Semantic segmentation input

## Setup imports

In [1]:
from auxiliary.nifti.io import read_nifti
from panoptica import (ConnectedComponentsInstanceApproximator,
                       NaiveOneToOneMatching, Panoptic_Evaluator, SemanticPair)

## Load data

In [2]:
ref_masks = read_nifti(
    "./spine_seg/semantic/ref.nii.gz"
)
pred_masks = read_nifti(
    "./spine_seg/semantic/pred.nii.gz"
)

## Run evaluation

In [3]:
sample = SemanticPair(pred_masks, ref_masks)

evaluator = Panoptic_Evaluator(
    expected_input=SemanticPair,
    instance_approximator=ConnectedComponentsInstanceApproximator(),
    instance_matcher=NaiveOneToOneMatching(),
    iou_threshold=0.5,
)

result, debug_data = evaluator.evaluate(sample)
print(result)

evaluate took 7.57350492477417 seconds to execute.
Number of instances in prediction: 89
Number of instances in reference: 87
True Positives (tp): 73
False Positives (fp): 16
False Negatives (fn): 14
Recognition Quality / F1 Score (RQ): 0.8295454545454546
Segmentation Quality (SQ): 0.7940127477906024 ± 0.11547745015679488
Panoptic Quality (PQ): 0.6586696657808406
volumetric instance-wise DICE: 0.8802182546605446 ± 0.07728416427007168
